In [45]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, random_split
from torchvision import datasets, transforms
from sklearn.utils.class_weight import compute_class_weight
import numpy as np
import os

# Constants
RANDOM_SEED = 42
BATCH_SIZE = 64
LEARNING_RATE = 0.00001
EPOCHS = 10
NUM_CLASSES = 3  # Adjust based on your classes
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [46]:
class AddGaussianNoise(object):
    def __init__(self, mean=0.0, std=0.05):
        self.mean = mean
        self.std = std

    def __call__(self, tensor):
        return tensor + torch.randn(tensor.size()) * self.std + self.mean
# Transforms
transform_train = transforms.Compose([
    transforms.Grayscale(num_output_channels=1),
    transforms.Resize((48, 48)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(20),
    transforms.RandomAffine(degrees=0, translate=(0.1, 0.1), scale=(0.9, 1.1)),
    transforms.ColorJitter(brightness=0.2, contrast=0.2),
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])
transform_test = transforms.Compose([
    transforms.Grayscale(num_output_channels=1),
    transforms.Resize((48, 48)),
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])
# Load dataset
dataset = datasets.ImageFolder(root="cnn_data", transform=transform_train)
# Train/val/test split: 70% train, 15% val, 15% test
train_size = int(0.7 * len(dataset))
temp_size = len(dataset) - train_size
train_dataset, temp_dataset = random_split(dataset, [train_size, temp_size], generator=torch.Generator().manual_seed(RANDOM_SEED))
# Split temp into val/test
val_size = temp_size // 2
test_size = temp_size - val_size  # Ensure full split
val_dataset, test_dataset = random_split(temp_dataset, [val_size, test_size], generator=torch.Generator().manual_seed(RANDOM_SEED))
# Set correct transforms for val/test
val_dataset.dataset.transform = transform_test
test_dataset.dataset.transform = transform_test
# Dataloaders
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)
# Compute class weights
labels = [sample[1] for sample in dataset.samples]
class_weights = compute_class_weight(class_weight='balanced', classes=np.unique(labels), y=labels)
class_weights = torch.tensor(class_weights, dtype=torch.float).to(DEVICE)

In [47]:
# CNN Model
class CNNModel(nn.Module):
    def __init__(self, output_size=NUM_CLASSES):
        super(CNNModel, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(1, 8, kernel_size=3, padding=1),
            nn.BatchNorm2d(8),
            nn.ReLU(),
            nn.MaxPool2d(2),

            nn.Conv2d(8, 16, kernel_size=3, padding=1),
            nn.BatchNorm2d(16),
            nn.ReLU(),
            nn.MaxPool2d(2),

            nn.Conv2d(16, 32, kernel_size=3, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )
        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(32 * 6 * 6, 64),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(32, output_size)
        )

    def forward(self, x):
        x = self.features(x)
        x = self.classifier(x)
        return x

# Model, loss, optimizer
model = CNNModel().to(DEVICE)
criterion = nn.CrossEntropyLoss(weight=class_weights, label_smoothing=0.1)
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE, weight_decay=1e-4)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.5)

# Early stopping parameters
best_val_loss = float('inf')
early_stop_counter = 0
patience = 7  # Early stopping patience

# Training loop
for epoch in range(EPOCHS):
    model.train()
    running_loss, correct, total = 0.0, 0, 0

    for images, labels in train_loader:
        images, labels = images.to(DEVICE), labels.to(DEVICE)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * images.size(0)
        _, predicted = torch.max(outputs, 1)
        correct += (predicted == labels).sum().item()
        total += labels.size(0)

    train_loss = running_loss / total
    train_acc = correct / total

    # Validation
    model.eval()
    val_loss, val_correct, val_total = 0.0, 0, 0
    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(DEVICE), labels.to(DEVICE)
            outputs = model(images)
            loss = criterion(outputs, labels)
            val_loss += loss.item() * images.size(0)
            _, predicted = torch.max(outputs, 1)
            val_correct += (predicted == labels).sum().item()
            val_total += labels.size(0)

    val_loss /= val_total
    val_acc = val_correct / val_total

    print(f"Epoch [{epoch+1}/{EPOCHS}] | "
          f"Train Loss: {train_loss:.4f} | Train Acc: {train_acc*100:.2f}% | "
          f"Val Loss: {val_loss:.4f} | Val Acc: {val_acc*100:.2f}%")

    # Save best model
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        early_stop_counter = 0
        torch.save(model.state_dict(), "best_vibration_model.pth")
        print("✅ Saved best model")
    else:
        early_stop_counter += 1
        if early_stop_counter >= patience:
            print("⏹️ Early stopping triggered.")
            break

    scheduler.step()

# Test the model
model.load_state_dict(torch.load("best_vibration_model.pth"))
model.eval()
test_loss, test_correct, test_total = 0.0, 0, 0
with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(DEVICE), labels.to(DEVICE)
        outputs = model(images)
        loss = criterion(outputs, labels)
        test_loss += loss.item() * images.size(0)
        _, predicted = torch.max(outputs, 1)
        test_correct += (predicted == labels).sum().item()
        test_total += labels.size(0)

test_loss /= test_total
test_acc = test_correct / test_total
print(f"Test Loss: {test_loss:.4f} | Test Acc: {test_acc*100:.2f}%")

Epoch [1/10] | Train Loss: 1.0859 | Train Acc: 40.21% | Val Loss: 1.0527 | Val Acc: 52.31%
✅ Saved best model
Epoch [2/10] | Train Loss: 1.0431 | Train Acc: 48.28% | Val Loss: 0.9980 | Val Acc: 58.06%
✅ Saved best model
Epoch [3/10] | Train Loss: 0.9920 | Train Acc: 55.43% | Val Loss: 0.9322 | Val Acc: 69.81%
✅ Saved best model
Epoch [4/10] | Train Loss: 0.9397 | Train Acc: 61.08% | Val Loss: 0.8677 | Val Acc: 79.42%
✅ Saved best model
Epoch [5/10] | Train Loss: 0.8931 | Train Acc: 65.73% | Val Loss: 0.8114 | Val Acc: 84.55%
✅ Saved best model
Epoch [6/10] | Train Loss: 0.8483 | Train Acc: 69.95% | Val Loss: 0.7584 | Val Acc: 86.76%
✅ Saved best model
Epoch [7/10] | Train Loss: 0.8051 | Train Acc: 73.47% | Val Loss: 0.7077 | Val Acc: 89.63%
✅ Saved best model
Epoch [8/10] | Train Loss: 0.7602 | Train Acc: 76.67% | Val Loss: 0.6536 | Val Acc: 91.49%
✅ Saved best model
Epoch [9/10] | Train Loss: 0.7170 | Train Acc: 79.74% | Val Loss: 0.6094 | Val Acc: 92.55%
✅ Saved best model
Epoch [10/

C:\Users\mohamed ahmed\AppData\Local\Temp\ipykernel_28824\2363172214.py:104: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load("best_vibration_m

Test Loss: 0.5611 | Test Acc: 94.37%


In [48]:
# Evaluate on Test Set
model = CNNModel().to(DEVICE)
criterion = nn.CrossEntropyLoss(weight=class_weights)
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.5)
model.load_state_dict(torch.load("best_vibration_model.pth"))
model.eval()
test_correct, test_total = 0, 0
with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(DEVICE), labels.to(DEVICE)
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        test_correct += (predicted == labels).sum().item()
        test_total += labels.size(0)

test_acc = test_correct / test_total
print(f"\n🎯 Final Test Accuracy: {test_acc * 100:.2f}%")

C:\Users\mohamed ahmed\AppData\Local\Temp\ipykernel_28824\2592146881.py:6: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load("best_vibration_mod


🎯 Final Test Accuracy: 93.54%
